### Imports, Setup

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import lightning as L
from dvclive import Live
from dvclive.lightning import DVCLiveLogger

# Data Preprocessing & Filtering
## INS-W_1 
### Feature Data
#### Bluetooth

In [52]:
def preprocessCSV(csv_file):
  
    df = pd.read_csv(csv_file) # read csv file
  
    df_filt = df.filter(regex=(".*14dhist|.*7dhist")) # filter column
    df_filt.dropna(inplace=True)
  
    df_str = df_filt.select_dtypes(object) # select only object columns, copy to new dataframe
  
    df_str = df_str.where(df_str == 'l', 0.0)
    df_str = df_str.where(df_str == 'm', 0.5)
    df_str = df_str.where(df_str == 'h', 1.0)
     
    df_str = df_str.astype(float) # convert to float


    df_max = df_filt.select_dtypes(include=['float64']) # select only float columns

    for col in df_max.columns:
        df_max[col] = df_max[col] / df_max[col].abs().max() # min max scaling

    df = pd.concat([df_max, df_str], axis=1).reindex(df_max.index) # combine the two dataframes
    del df_max, df_str # delete the temporary dataframes
    
    return df

In [53]:
df_W1_BT = preprocessCSV('csv_data/INS-W_1/FeatureData/bluetooth.csv')
df_W1_Call = preprocessCSV("csv_data/INS-W_1/FeatureData/call.csv")
df_W1_Loc = preprocessCSV("csv_data/INS-W_1/FeatureData/location.csv")
df_W1_Screen = preprocessCSV("csv_data/INS-W_1/FeatureData/screen.csv")
df_W1_Sleep = preprocessCSV("csv_data/INS-W_1/FeatureData/sleep.csv")
df_W1_Steps = preprocessCSV("csv_data/INS-W_1/FeatureData/steps.csv")
df_W1_WiFi = preprocessCSV("csv_data/INS-W_1/FeatureData/wifi.csv")

df_W1_WiFi.head()

C:\Users\darkenral\AppData\Local\Temp\ipykernel_63552\1004289291.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt.dropna(inplace=True)
C:\Users\darkenral\AppData\Local\Temp\ipykernel_63552\1004289291.py:3: DtypeWarning: Columns (556,557,565) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file) # read csv file
C:\Users\darkenral\AppData\Local\Temp\ipykernel_63552\1004289291.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt.dropna(inplace=True)
C:\Users\darkenral\AppData\Local\Temp\ipykernel_63552\1004289291.py:6: SettingWithCopyWarning: 
A value is t

,f_wifi:phone_wifi_connected_rapids_countscans:14dhist,f_wifi:phone_wifi_connected_rapids_uniquedevices:14dhist,f_wifi:phone_wifi_connected_rapids_countscansmostuniquedevice:14dhist,f_wifi:phone_wifi_connected_rapids_countscans_norm:14dhist,f_wifi:phone_wifi_connected_rapids_uniquedevices_norm:14dhist,f_wifi:phone_wifi_connected_rapids_countscansmostuniquedevice_norm:14dhist,f_wifi:phone_wifi_connected_rapids_countscans:7dhist,f_wifi:phone_wifi_connected_rapids_uniquedevices:7dhist,f_wifi:phone_wifi_connected_rapids_countscansmostuniquedevice:7dhist,f_wifi:phone_wifi_connected_rapids_countscans_norm:7dhist,f_wifi:phone_wifi_connected_rapids_uniquedevices_norm:7dhist,f_wifi:phone_wifi_connected_rapids_countscansmostuniquedevice_norm:7dhist,f_wifi:phone_wifi_connected_rapids_countscans_dis:14dhist,f_wifi:phone_wifi_connected_rapids_uniquedevices_dis:14dhist,f_wifi:phone_wifi_connected_rapids_countscansmostuniquedevice_dis:14dhist,f_wifi:phone_wifi_connected_rapids_countscans_dis:7dhist,f_wifi:phone_wifi_connected_rapids_uniquedevices_dis:7dhist,f_wifi:phone_wifi_connected_rapids_countscansmostuniquedevice_dis:7dhist
15,0.002841,0.010292,0.001941,-0.216938,-0.069990,-0.473001,0.005492,0.015707,0.003591,-0.033177,0.015961,-0.242358,1.0,1.0,1.0,1.0,1.0,1.0
16,0.006188,0.018868,0.004692,-0.189916,0.000000,-0.373112,0.011960,0.028796,0.008677,-0.005266,0.095767,-0.122064,1.0,1.0,1.0,1.0,1.0,1.0
17,0.006188,0.018868,0.004692,-0.189916,0.000000,-0.373112,0.011960,0.028796,0.008677,-0.005266,0.095767,-0.122064,1.0,1.0,1.0,1.0,1.0,1.0
18,0.007135,0.018868,0.005905,-0.182268,0.000000,-0.329044,0.013791,0.028796,0.010922,0.002633,0.095767,-0.068992,1.0,1.0,1.0,1.0,1.0,1.0
19,0.019953,0.025729,0.010193,-0.078771,0.055992,-0.173336,0.038565,0.039267,0.018851,0.109538,0.159611,0.118526,1.0,1.0,1.0,1.0,1.0,1.0


In [22]:
df_W1_BT = pd.read_csv('csv_data/INS-W_1/FeatureData/bluetooth.csv')


df_W1_BT_filt = df_W1_BT.filter(regex=(".*14dhist|.*7dhist")) # filter columns
df_W1_BT_filt.dropna(inplace=True)

df_W1_BT_mxscale_str = df_W1_BT_filt.select_dtypes(object) # select only object columns, copy to new dataframe

for col in df_W1_BT_mxscale_str.columns:
    df_W1_BT_mxscale_str[col] = df_W1_BT_mxscale_str[col].replace({'l': 0.0, 'm': 0.5, 'h': 1.0}) # low, medium, high normalised to 0, 0.5, 1

df_W1_BT_mxscale_str = df_W1_BT_mxscale_str.astype(float) # convert to float


df_W1_BT_mxscale = df_W1_BT_filt.select_dtypes(include=['float64']) # select only float columns

for col in df_W1_BT_mxscale.columns:
    df_W1_BT_mxscale[col] = df_W1_BT_mxscale[col] / df_W1_BT_mxscale[col].abs().max() # min max scaling

df_W1_BT = pd.concat([df_W1_BT_mxscale, df_W1_BT_mxscale_str], axis=1).reindex(df_W1_BT_mxscale.index) # combine the two dataframes
del df_W1_BT_mxscale, df_W1_BT_mxscale_str # delete the temporary dataframes

df_W1_BT.head()

C:\Users\darkenral\AppData\Local\Temp\ipykernel_63552\1051641938.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_W1_BT_filt.dropna(inplace=True)
C:\Users\darkenral\AppData\Local\Temp\ipykernel_63552\1051641938.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_W1_BT_mxscale_str[col] = df_W1_BT_mxscale_str[col].replace({'l': 0.0, 'm': 0.5, 'h': 1.0}) # low, medium, high normalised to 0, 0.5, 1


,f_blue:phone_bluetooth_rapids_countscans:14dhist,f_blue:phone_bluetooth_rapids_uniquedevices:14dhist,f_blue:phone_bluetooth_rapids_countscansmostuniquedevice:14dhist,f_blue:phone_bluetooth_doryab_countscansall:14dhist,f_blue:phone_bluetooth_doryab_uniquedevicesall:14dhist,f_blue:phone_bluetooth_doryab_meanscansall:14dhist,f_blue:phone_bluetooth_doryab_stdscansall:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdevicewithinsegmentsall:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrosssegmentsall:14dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrossdatasetall:14dhist,...,f_blue:phone_bluetooth_doryab_countscansothers_dis:7dhist,f_blue:phone_bluetooth_doryab_uniquedevicesothers_dis:7dhist,f_blue:phone_bluetooth_doryab_meanscansothers_dis:7dhist,f_blue:phone_bluetooth_doryab_stdscansothers_dis:7dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdevicewithinsegmentsothers_dis:7dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrosssegmentsothers_dis:7dhist,f_blue:phone_bluetooth_doryab_countscansmostfrequentdeviceacrossdatasetothers_dis:7dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdevicewithinsegmentsothers_dis:7dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrosssegmentsothers_dis:7dhist,f_blue:phone_bluetooth_doryab_countscansleastfrequentdeviceacrossdatasetothers_dis:7dhist
14,0.006377,0.009792,0.003788,0.006377,0.009792,0.049827,0.010688,0.003788,0.003788,0.003788,...,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5
15,0.023622,0.029994,0.006944,0.023622,0.029994,0.060250,0.015546,0.006944,0.006944,0.006944,...,0.5,0.5,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5
16,0.031468,0.041022,0.017045,0.031468,0.041022,0.058684,0.019361,0.017045,0.017045,0.017045,...,0.5,0.5,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5
17,0.031468,0.041022,0.017045,0.031468,0.041022,0.058684,0.019361,0.017045,0.017045,0.017045,...,0.5,0.5,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5
18,0.031510,0.041022,0.017045,0.031510,0.041022,0.058763,0.019389,0.017045,0.017045,0.017045,...,0.5,0.5,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5
